testing on my Windows laptop

In [1]:
import matplotlib.pyplot as plt
from databroker import Broker
import snapshot

from bluesky.utils import install_nb_kicker  # for notebooks, qt -> nb
install_nb_kicker()
plt.ion()

from bluesky import RunEngine
RE = RunEngine({})

mongo_host = "poof"
cfg = {
    'metadatastore' : {
        'module' : 'databroker.headersource.mongo',
        'class' : 'MDS',
        'config' : {
            'host': mongo_host,
             'port': 27017,
             'database': 'metadatastore-production-v1',
             'timezone': 'US/Central'},
    },
    'assets' : {
        'module' : 'databroker.assets.mongo',
        'class' : 'Registry',
        'config': {
            'host': mongo_host, 
            'port': 27017, 
            'database': 'filestore-production-v1'}
    },
}
db = Broker.from_config(cfg)
RE.subscribe(db.insert)

0

In [2]:
#h = db[-1]
#h.start

In [3]:
from bluesky import plans as bp
from bluesky import callbacks as bc
from ophyd import Signal, EpicsSignal

reporter = snapshot.SnapshotReport()
signal = Signal(name="signal")

In [4]:
RE(bp.count([signal]), bc.LiveTable([signal]))
h = db[-1]
h.table()

+-----------+------------+------------+
|   seq_num |       time |     signal |
+-----------+------------+------------+
|         1 | 00:08:46.7 |      0.000 |
+-----------+------------+------------+
generator count ['9d52f3ce'] (scan num: 1)


,time,signal
seq_num,,
1,2018-12-20 00:08:46.790509462,0.0


In [5]:
RE(snapshot.snapshot([signal], md=dict(purpose="non-EPICS data")), bc.LiveTable([signal]))
h = db[-1]
reporter.print_report(h)
h.table()

+-----------+------------+------------+
|   seq_num |       time |     signal |
+-----------+------------+------------+
|         1 | 00:08:47.2 |      0.000 |
+-----------+------------+------------+
generator snapshot ['a97501e0'] (scan num: 2)
snapshot: 2018-12-20 00:08:47.182512

hints: {}
iso8601: 2018-12-20 00:08:47.182512
plan_description: archive snapshot of ophyd Signals (usually EPICS PVs)
plan_name: snapshot
plan_type: generator
purpose: non-EPICS data
scan_id: 2
software_versions: {'PYTHON': '3.6.7 |Anaconda, Inc.| (default, Dec 10 2018, 20:35:02) [MSC v.1915 64 bit (AMD64)]', 'PYEPICS': '3.3.1', 'BLUESKY': '1.3.3.post17+g50b44c4', 'OPHYD': '1.2.0.post19+g56ba9fd', 'DATABROKER': '0.11.3', 'APS_BLUESKY_TOOLS': '0.0.35+10.g3276dd6'}
time: 1545286127.1825125
uid: a97501e0-ffc7-4454-b9ff-0d829ce7592d

====== ====== ===== ==========================
source name   value timestamp                 
====== ====== ===== ==========================
SIM    signal 0.0   2018-12-20 00:08:46

,time,signal
seq_num,,
1,2018-12-20 00:08:47.220919371,0.0


In [6]:
pvlist = "xxx:SR_heartbeat  xxx:HEARTBEAT".split()
obj_dict = snapshot.connect_pvlist(pvlist)
RE(snapshot.snapshot(obj_dict.values(), md=dict(purpose="EPICS data")), bc.LiveTable(obj_dict.values()))
h = db[-1]
reporter.print_report(h)
h.table()

+-----------+------------+------------+------------+
|   seq_num |       time |   signal_0 |   signal_1 |
+-----------+------------+------------+------------+
|         1 | 00:08:47.7 |          0 |       5281 |
+-----------+------------+------------+------------+
generator snapshot ['c1d1cf4a'] (scan num: 3)
snapshot: 2018-12-20 00:08:47.650883

hints: {}
iso8601: 2018-12-20 00:08:47.650883
plan_description: archive snapshot of ophyd Signals (usually EPICS PVs)
plan_name: snapshot
plan_type: generator
purpose: EPICS data
scan_id: 3
software_versions: {'PYTHON': '3.6.7 |Anaconda, Inc.| (default, Dec 10 2018, 20:35:02) [MSC v.1915 64 bit (AMD64)]', 'PYEPICS': '3.3.1', 'BLUESKY': '1.3.3.post17+g50b44c4', 'OPHYD': '1.2.0.post19+g56ba9fd', 'DATABROKER': '0.11.3', 'APS_BLUESKY_TOOLS': '0.0.35+10.g3276dd6'}
time: 1545286127.650883
uid: c1d1cf4a-5d56-4a0a-958d-e0869e9a1492

====== ================ ====== ==========================
source name             value  timestamp                 
====

,time,signal_1,signal_0
seq_num,,,
1,2018-12-20 00:08:47.797491550,5281.0,0
